In [30]:
%matplotlib inline
import numpy as np
import torch
from torch.utils.serialization import load_lua
import os
import scipy.io as sio
import cv2
import math
from matplotlib import pyplot as plt

import sqlite3

In [31]:
#Change this paths according to your directories
images_path = "/Data/nruiz9/data/facial_landmarks/AFLW/aflw/data/flickr/"
storing_path = "/Data/nruiz9/data/facial_landmarks/AFLW/aflw_cropped_loose_test/"

In [32]:
#Load KEPLER split file
test_set = sio.loadmat('/Data/nruiz9/data/facial_landmarks/AFLW/testset.mat')
print test_set['test'][0][0][0].shape
print test_set['test'][0][0][1].shape
print test_set['test'][0][0][2].shape
print len(test_set['test'][0][0])

id_test_set = np.concatenate([test_set['test'][0][0][0], test_set['test'][0][0][1], test_set['test'][0][0][2]])
print id_test_set.shape

# I will just use every single one for now. If results are not good I'll sample equally.

(573, 1)
(232, 1)
(165, 1)
3
(970, 1)


In [38]:
#Image counter
counter = 0

#Open the sqlite database
conn = sqlite3.connect('/Data/nruiz9/data/facial_landmarks/AFLW/aflw/data/aflw.sqlite')
c = conn.cursor()

#Creating the query string for retriving: roll, pitch, yaw and faces position
#Change it according to what you want to retrieve
select_string = "faceimages.filepath, faces.face_id, facepose.roll, facepose.pitch, facepose.yaw, facerect.x, facerect.y, facerect.w, facerect.h"
from_string = "faceimages, faces, facepose, facerect"
where_string = "faces.face_id = facepose.face_id and faces.file_id = faceimages.file_id and faces.face_id = facerect.face_id"
query_string = "SELECT " + select_string + " FROM " + from_string + " WHERE " + where_string

test_file_txt = '/Data/nruiz9/data/facial_landmarks/AFLW/KEPLER_test_split.txt'
out_txt = open(test_file_txt, 'w')
#It iterates through the rows returned from the query
for row in c.execute(query_string):

    #Using our specific query_string, the "row" variable will contain:
    # row[0] = image path
    # row[1] = face id
    # row[2] = rollgma
    # row[3] = pitch
    # row[4] = yaw
    # row[5] = face coord x
    # row[6] = face coord y
    # row[7] = face width
    # row[8] = face heigh
    if row[1] in id_test_set:
        #Creating the full path names for input and output
        input_path = images_path + str(row[0])
        output_path = storing_path + str(row[0])

        #If the file exist then open it       
        if(os.path.isfile(input_path)  == True):

            out_txt.write(str(row[0]) + '.jpg\n')
            #Increasing the counter
            counter = counter + 1 

        #if the file does not exits it return an exception
        else:
            raise ValueError('Error: I cannot find the file specified: ' + str(input_path))

#Once finished the iteration it closes the database
print counter
out_txt.close()
c.close()
print 'Done'

970
Done


In [4]:
print 'test'

test
